# 🚀 PyPozo 2.0 - Demo Completo del Módulo

## 🎯 Guía Completa de Uso del Módulo PyPozo

**Bienvenido al tutorial completo de PyPozo 2.0!** 🔥

Este notebook demuestra todas las funcionalidades principales del módulo PyPozo para análisis petrofísico y visualización de pozos petroleros.

### 📋 **¿Qué aprenderás?**

- ✅ **Importar y configurar** PyPozo
- ✅ **Cargar archivos LAS** de pozos petroleros  
- ✅ **Fusionar pozos duplicados** automáticamente
- ✅ **Calcular propiedades petrofísicas** (VCL, Porosidad, Saturación)
- ✅ **Visualizar curvas** con gráficos profesionales
- ✅ **Exportar resultados** en múltiples formatos

### 🔧 **Requisitos**
- Python 3.8+
- PyPozo 2.0 instalado
- Archivos LAS de ejemplo (incluidos en `/data/`)

---

**Autor**: PyPozo Team  
**Fecha**: Julio 3, 2025  
**Versión**: 2.0.0

## 1. 📦 Importar Librerías y Módulos Principales

Primero, importamos todas las librerías necesarias y verificamos que PyPozo esté instalado correctamente.

In [ ]:
# Imports básicos
import sys
import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Configurar matplotlib para notebook
%matplotlib inline
plt.style.use('seaborn-v0_8')

print("🔍 Importando módulos de PyPozo...")

# Imports de PyPozo
try:
    from src.pypozo.core.well import WellManager, WellDataFrame
    from src.pypozo.visualization.plotter import WellPlotter
    from src.pypozo.petrophysics import VclCalculator, PorosityCalculator
    from src.pypozo.petrophysics import WaterSaturationCalculator, PermeabilityCalculator, LithologyAnalyzer
    
    print("✅ Módulos de PyPozo importados exitosamente!")
    
except ImportError as e:
    print(f"❌ Error importando PyPozo: {e}")
    print("💡 Asegúrate de ejecutar el notebook desde el directorio raíz del proyecto")

# Verificar versión
try:
    import src.pypozo
    print(f"📋 PyPozo versión: 2.0.0")
    print(f"🐍 Python versión: {sys.version}")
    print(f"📁 Directorio de trabajo: {os.getcwd()}")
except:
    print("⚠️ No se pudo determinar la versión de PyPozo")

## 2. 📁 Cargar Archivos LAS de Ejemplo

PyPozo puede cargar archivos LAS (Log ASCII Standard) que contienen datos de registros de pozos petroleros. Vamos a cargar algunos pozos de ejemplo.

In [ ]:
# Configurar rutas de datos
data_path = Path("../data")  # Desde notebooks hacia data
print(f"📂 Buscando archivos LAS en: {data_path.absolute()}")

# Buscar archivos LAS disponibles
las_files = list(data_path.glob("*.las"))
print(f"🔍 Archivos LAS encontrados: {len(las_files)}")

# Mostrar archivos disponibles
for i, file in enumerate(las_files[:5], 1):  # Mostrar solo los primeros 5
    print(f"  {i}. {file.name}")

if len(las_files) > 5:
    print(f"  ... y {len(las_files) - 5} más")

# Seleccionar algunos pozos de ejemplo para el demo
pozos_ejemplo = []
archivos_demo = ["ABEDUL-1_MERGED.las", "ARIEL-1_MERGED.las", "CABALLO-1_MERGED.las"]

print("\n🚀 Cargando pozos de ejemplo...")

for archivo in archivos_demo:
    archivo_path = data_path / archivo
    if archivo_path.exists():
        try:
            print(f"📖 Cargando: {archivo}")
            well = WellManager.load_from_las(str(archivo_path))
            pozos_ejemplo.append((archivo.replace("_MERGED.las", ""), well))
            print(f"  ✅ {archivo} cargado exitosamente")
            print(f"     - Curvas: {len(well.curves)}")
            print(f"     - Rango: {well.depth_range[0]:.1f} - {well.depth_range[1]:.1f} m")
        except Exception as e:
            print(f"  ❌ Error cargando {archivo}: {e}")
    else:
        print(f"  ⚠️ Archivo no encontrado: {archivo}")

print(f"\n🎯 Total de pozos cargados: {len(pozos_ejemplo)}")

# Si no hay pozos, intentar cargar cualquier archivo disponible
if not pozos_ejemplo and las_files:
    print("\n🔄 Intentando cargar el primer archivo disponible...")
    try:
        first_file = las_files[0]
        well = WellManager.load_from_las(str(first_file))
        pozos_ejemplo.append((first_file.stem, well))
        print(f"✅ Cargado: {first_file.name}")
    except Exception as e:
        print(f"❌ Error: {e}")

## 3. 📊 Visualizar Pozos Cargados

Ahora vamos a examinar los pozos que hemos cargado, mostrando sus metadatos y curvas disponibles.

In [ ]:
# Crear tabla resumen de pozos cargados
if pozos_ejemplo:
    print("📋 RESUMEN DE POZOS CARGADOS")
    print("=" * 60)
    
    # Crear DataFrame con información de pozos
    pozos_info = []
    
    for nombre, well in pozos_ejemplo:
        depth_range = well.depth_range
        pozos_info.append({
            'Pozo': nombre,
            'Curvas': len(well.curves),
            'Prof_Min (m)': f"{depth_range[0]:.1f}",
            'Prof_Max (m)': f"{depth_range[1]:.1f}",
            'Intervalo (m)': f"{depth_range[1] - depth_range[0]:.1f}",
            'Puntos': len(well.data) if hasattr(well, 'data') else 'N/A'
        })
    
    df_pozos = pd.DataFrame(pozos_info)
    print(df_pozos.to_string(index=False))
    
    # Mostrar curvas disponibles para el primer pozo
    if pozos_ejemplo:
        primer_pozo_nombre, primer_pozo = pozos_ejemplo[0]
        print(f"\n🔍 DETALLE DEL POZO: {primer_pozo_nombre}")
        print("-" * 40)
        print(f"📊 Curvas disponibles ({len(primer_pozo.curves)}):")
        
        # Organizar curvas por tipo
        curvas_basicas = ['GR', 'SP', 'CAL', 'RT', 'RHOB', 'NPHI']
        curvas_encontradas_basicas = [c for c in primer_pozo.curves if c in curvas_basicas]
        curvas_otras = [c for c in primer_pozo.curves if c not in curvas_basicas]
        
        if curvas_encontradas_basicas:
            print(f"  🔧 Básicas: {', '.join(curvas_encontradas_basicas)}")
        if curvas_otras:
            print(f"  📈 Otras: {', '.join(curvas_otras[:10])}")  # Mostrar solo las primeras 10
            if len(curvas_otras) > 10:
                print(f"       ... y {len(curvas_otras) - 10} más")
        
        # Mostrar estadísticas básicas de una curva (GR si está disponible)
        if 'GR' in primer_pozo.curves:
            gr_data = primer_pozo.get_curve_data('GR')
            if gr_data is not None and len(gr_data) > 0:
                print(f"\n📈 ESTADÍSTICAS DE GR:")
                print(f"  • Promedio: {gr_data.mean():.1f} API")
                print(f"  • Mínimo: {gr_data.min():.1f} API")
                print(f"  • Máximo: {gr_data.max():.1f} API")
                print(f"  • Desv. Std: {gr_data.std():.1f} API")

else:
    print("❌ No se cargaron pozos para mostrar")

## 4. 🔗 Fusión de Pozos Duplicados

Una característica única de PyPozo 2.0 es la capacidad de fusionar pozos automáticamente. Esto es útil cuando tienes múltiples archivos del mismo pozo con diferentes registros.

In [ ]:
# Demo de fusión de pozos
print("🔗 DEMOSTRACIÓN DE FUSIÓN DE POZOS")
print("=" * 50)

if len(pozos_ejemplo) >= 2:
    # Seleccionar dos pozos para fusionar
    pozo1_nombre, pozo1 = pozos_ejemplo[0]
    pozo2_nombre, pozo2 = pozos_ejemplo[1]
    
    print(f"📊 Pozos a fusionar:")
    print(f"  1. {pozo1_nombre}: {len(pozo1.curves)} curvas")
    print(f"  2. {pozo2_nombre}: {len(pozo2.curves)} curvas")
    
    # Intentar fusión
    try:
        print(f"\n🔄 Ejecutando fusión...")
        
        # Usar el método de fusión de WellDataFrame
        pozos_para_fusion = [pozo1, pozo2]
        nombre_fusionado = f"FUSION_{pozo1_nombre}_{pozo2_nombre}"
        
        pozo_fusionado = WellDataFrame.merge_wells(pozos_para_fusion, nombre_fusionado)
        
        if pozo_fusionado:
            print(f"✅ Fusión exitosa!")
            print(f"📋 Pozo fusionado: {nombre_fusionado}")
            print(f"📊 Curvas totales: {len(pozo_fusionado.curves)}")
            
            # Comparar rangos de profundidad
            range1 = pozo1.depth_range
            range2 = pozo2.depth_range
            range_fusionado = pozo_fusionado.depth_range
            
            print(f"\n📏 COMPARACIÓN DE RANGOS:")
            print(f"  {pozo1_nombre}: {range1[0]:.1f} - {range1[1]:.1f} m")
            print(f"  {pozo2_nombre}: {range2[0]:.1f} - {range2[1]:.1f} m")
            print(f"  Fusionado: {range_fusionado[0]:.1f} - {range_fusionado[1]:.1f} m")
            
            # Agregar pozo fusionado a la lista
            pozos_ejemplo.append((nombre_fusionado, pozo_fusionado))
            
        else:
            print("❌ Error en la fusión")
            
    except Exception as e:
        print(f"❌ Error durante la fusión: {e}")
        print("💡 La fusión puede fallar si los pozos no tienen curvas compatibles")

else:
    print("⚠️ Se necesitan al menos 2 pozos para demostrar la fusión")
    print("🔧 Creando pozos sintéticos para la demo...")
    
    # Crear pozos sintéticos para demostrar la fusión
    if len(pozos_ejemplo) >= 1:
        pozo_base = pozos_ejemplo[0][1]
        
        # Simular un segundo pozo con parte de los datos del primero
        print("📊 Creando demo sintético de fusión...")
        print("💡 En un caso real, tendrías pozos con traslapes naturales")

print(f"\n🎯 Total de pozos disponibles ahora: {len(pozos_ejemplo)}")

## 5. 🧮 Cálculos Petrofísicos Básicos (VCL y Porosidad)

PyPozo incluye calculadoras especializadas para propiedades petrofísicas fundamentales. Vamos a calcular el Volumen de Arcilla (VCL) y la Porosidad Efectiva (PHIE).

In [ ]:
# Inicializar calculadoras petrofísicas
vcl_calc = VclCalculator()
porosity_calc = PorosityCalculator()

print("🧮 CÁLCULOS PETROFÍSICOS")
print("=" * 40)

if pozos_ejemplo:
    # Seleccionar el primer pozo para los cálculos
    pozo_nombre, pozo = pozos_ejemplo[0]
    print(f"📊 Usando pozo: {pozo_nombre}")
    
    # === CÁLCULO DE VCL (Volumen de Arcilla) ===
    print(f"\n🏔️ CÁLCULO DE VCL")
    print("-" * 25)
    
    # Buscar curva de Rayos Gamma
    gr_curves = [c for c in pozo.curves if 'GR' in c.upper()]
    
    if gr_curves:
        gr_curve = gr_curves[0]
        print(f"📈 Usando curva GR: {gr_curve}")
        
        try:
            # Obtener datos de GR
            gr_data = pozo.get_curve_data(gr_curve)
            
            if gr_data is not None and len(gr_data) > 0:
                # Parámetros para VCL
                gr_min = 15  # API (arenisca limpia)
                gr_max = 150  # API (arcilla pura)
                method = "larionov_tertiary"  # Método de Larionov para rocas terciarias
                
                print(f"⚙️ Parámetros:")
                print(f"  • GR mínimo: {gr_min} API")
                print(f"  • GR máximo: {gr_max} API") 
                print(f"  • Método: {method}")
                
                # Calcular VCL
                result = vcl_calc.calculate(
                    gr_data=gr_data.values,
                    method=method,
                    gr_clean=gr_min,
                    gr_clay=gr_max
                )
                
                if result.get('success', False):
                    vcl_values = result['vcl']
                    print(f"✅ VCL calculado exitosamente!")
                    print(f"📊 Estadísticas VCL:")
                    print(f"  • Promedio: {np.nanmean(vcl_values):.3f}")
                    print(f"  • Mínimo: {np.nanmin(vcl_values):.3f}")
                    print(f"  • Máximo: {np.nanmax(vcl_values):.3f}")
                    
                    # Agregar VCL al pozo
                    vcl_curve_name = f"VCL_{method.upper()}"
                    success = pozo.add_curve(vcl_curve_name, vcl_values, units='v/v', 
                                           description=f'Volume of clay - {method} method')
                    if success:
                        print(f"💾 Curva {vcl_curve_name} agregada al pozo")
                    
                else:
                    print(f"❌ Error calculando VCL: {result.get('error', 'Error desconocido')}")
            else:
                print(f"❌ No hay datos válidos en la curva {gr_curve}")
                
        except Exception as e:
            print(f"❌ Error procesando VCL: {e}")
    else:
        print("⚠️ No se encontró curva GR para calcular VCL")
    
    # === CÁLCULO DE POROSIDAD ===
    print(f"\n🕳️ CÁLCULO DE POROSIDAD")
    print("-" * 30)
    
    # Buscar curvas necesarias
    rhob_curves = [c for c in pozo.curves if any(x in c.upper() for x in ['RHOB', 'DEN'])]
    nphi_curves = [c for c in pozo.curves if any(x in c.upper() for x in ['NPHI', 'NEU'])]
    
    if rhob_curves and nphi_curves:
        rhob_curve = rhob_curves[0]
        nphi_curve = nphi_curves[0]
        print(f"📈 Usando RHOB: {rhob_curve}")
        print(f"📈 Usando NPHI: {nphi_curve}")
        
        try:
            # Obtener datos
            rhob_data = pozo.get_curve_data(rhob_curve)
            nphi_data = pozo.get_curve_data(nphi_curve)
            
            if rhob_data is not None and nphi_data is not None:
                # Parámetros de porosidad
                matrix_density = 2.65  # g/cc (cuarzo)
                fluid_density = 1.0    # g/cc (agua)
                
                print(f"⚙️ Parámetros:")
                print(f"  • Densidad matriz: {matrix_density} g/cc")
                print(f"  • Densidad fluido: {fluid_density} g/cc")
                
                # Calcular porosidad combinada (densidad + neutrón)
                result = porosity_calc.calculate_density_neutron_porosity(
                    bulk_density=rhob_data.values,
                    neutron_porosity=nphi_data.values,
                    matrix_density=matrix_density,
                    fluid_density=fluid_density
                )
                
                if result.get('success', False):
                    phie_values = result['phie']
                    print(f"✅ Porosidad calculada exitosamente!")
                    print(f"📊 Estadísticas PHIE:")
                    print(f"  • Promedio: {np.nanmean(phie_values):.3f}")
                    print(f"  • Mínimo: {np.nanmin(phie_values):.3f}")
                    print(f"  • Máximo: {np.nanmax(phie_values):.3f}")
                    
                    # Agregar PHIE al pozo
                    phie_curve_name = "PHIE_COMBINED"
                    success = pozo.add_curve(phie_curve_name, phie_values, units='v/v',
                                           description='Effective porosity - combined method')
                    if success:
                        print(f"💾 Curva {phie_curve_name} agregada al pozo")
                        
                else:
                    print(f"❌ Error calculando porosidad: {result.get('error', 'Error desconocido')}")
            else:
                print("❌ No hay datos válidos en las curvas de densidad/neutrón")
                
        except Exception as e:
            print(f"❌ Error procesando porosidad: {e}")
            
    elif rhob_curves:
        print(f"📈 Solo RHOB disponible: {rhob_curves[0]}")
        print("💡 Calculando porosidad solo con densidad...")
        # Aquí podrías implementar cálculo solo con densidad
        
    elif nphi_curves:
        print(f"📈 Solo NPHI disponible: {nphi_curves[0]}")
        print("💡 Calculando porosidad solo con neutrón...")
        # Aquí podrías implementar cálculo solo con neutrón
        
    else:
        print("⚠️ No se encontraron curvas RHOB o NPHI para calcular porosidad")

else:
    print("❌ No hay pozos disponibles para cálculos petrofísicos")

## 6. 💧 Cálculo de Saturación de Agua

La saturación de agua es una propiedad crítica en la evaluación de reservorios. PyPozo implementa varios métodos incluyendo Archie y Simandoux.

In [ ]:
# Inicializar calculadora de saturación de agua
sw_calc = WaterSaturationCalculator()

print("💧 CÁLCULO DE SATURACIÓN DE AGUA")
print("=" * 45)

if pozos_ejemplo:
    pozo_nombre, pozo = pozos_ejemplo[0]
    print(f"📊 Usando pozo: {pozo_nombre}")
    
    # Buscar curvas necesarias
    rt_curves = [c for c in pozo.curves if any(x in c.upper() for x in ['RT', 'RES', 'ILD', 'LLD'])]
    phie_curves = [c for c in pozo.curves if 'PHIE' in c.upper()]
    vcl_curves = [c for c in pozo.curves if 'VCL' in c.upper()]
    
    print(f"\n🔍 Curvas disponibles:")
    print(f"  • Resistividad: {rt_curves[:3] if rt_curves else 'No encontrada'}")
    print(f"  • Porosidad: {phie_curves[:3] if phie_curves else 'No encontrada'}")
    print(f"  • VCL: {vcl_curves[:3] if vcl_curves else 'No encontrada'}")
    
    if rt_curves and phie_curves:
        rt_curve = rt_curves[0]
        phie_curve = phie_curves[0]
        
        print(f"\n📈 Usando:")
        print(f"  • RT: {rt_curve}")
        print(f"  • PHIE: {phie_curve}")
        
        try:
            # Obtener datos
            rt_data = pozo.get_curve_data(rt_curve)
            phie_data = pozo.get_curve_data(phie_curve)
            
            if rt_data is not None and phie_data is not None:
                # Parámetros de Archie
                a = 1.0      # Factor de tortuosidad
                m = 2.0      # Exponente de cementación  
                n = 2.0      # Exponente de saturación
                rw = 0.05    # Resistividad del agua (ohm-m)
                
                print(f"\n⚙️ Parámetros de Archie:")
                print(f"  • a (tortuosidad): {a}")
                print(f"  • m (cementación): {m}")
                print(f"  • n (saturación): {n}")
                print(f"  • Rw: {rw} ohm-m")
                
                # Calcular Sw usando método de Archie simple
                try:
                    result = sw_calc.calculate_archie_simple(
                        resistivity=rt_data.values,
                        porosity=phie_data.values,
                        a=a, m=m, n=n, rw=rw
                    )
                    
                    if result.get('success', False):
                        sw_values = result['sw']
                        print(f"\n✅ Saturación de agua calculada!")
                        print(f"📊 Estadísticas Sw:")
                        print(f"  • Promedio: {np.nanmean(sw_values):.3f}")
                        print(f"  • Mínimo: {np.nanmin(sw_values):.3f}")
                        print(f"  • Máximo: {np.nanmax(sw_values):.3f}")
                        
                        # Clasificar saturación
                        sw_mean = np.nanmean(sw_values)
                        if sw_mean > 0.8:
                            clasificacion = "🌊 Zona de agua"
                        elif sw_mean > 0.5:
                            clasificacion = "🔄 Zona de transición"
                        else:
                            clasificacion = "🛢️ Zona de hidrocarburo"
                        
                        print(f"🎯 Clasificación: {clasificacion}")
                        
                        # Agregar Sw al pozo
                        sw_curve_name = "SW_ARCHIE"
                        success = pozo.add_curve(sw_curve_name, sw_values, units='v/v',
                                               description='Water saturation - Archie method')
                        if success:
                            print(f"💾 Curva {sw_curve_name} agregada al pozo")
                    else:
                        print(f"❌ Error calculando Sw: {result.get('error', 'Error desconocido')}")
                        
                except Exception as e:
                    print(f"❌ Error en cálculo de Archie: {e}")
                    print("💡 Implementando cálculo simplificado...")
                    
                    # Cálculo simplificado de Archie si el método específico falla
                    valid_mask = (rt_data > 0) & (phie_data > 0.01) & np.isfinite(rt_data) & np.isfinite(phie_data)
                    
                    if np.any(valid_mask):
                        # Sw = ((a * Rw) / (phi^m * Rt))^(1/n)
                        rt_valid = rt_data[valid_mask]
                        phi_valid = phie_data[valid_mask]
                        
                        sw_simplified = ((a * rw) / (phi_valid**m * rt_valid))**(1/n)
                        sw_simplified = np.clip(sw_simplified, 0, 1)  # Limitar entre 0 y 1
                        
                        print(f"✅ Cálculo simplificado completado!")
                        print(f"📊 Estadísticas Sw (simplificado):")
                        print(f"  • Promedio: {np.mean(sw_simplified):.3f}")
                        print(f"  • Mínimo: {np.min(sw_simplified):.3f}")
                        print(f"  • Máximo: {np.max(sw_simplified):.3f}")
                    else:
                        print("❌ No hay datos válidos para el cálculo")
            else:
                print("❌ No hay datos válidos en las curvas de RT o PHIE")
                
        except Exception as e:
            print(f"❌ Error procesando saturación de agua: {e}")
    else:
        print("⚠️ Se necesitan curvas de resistividad y porosidad para calcular Sw")
        print("💡 Nota: Asegúrate de haber ejecutado el cálculo de porosidad primero")

else:
    print("❌ No hay pozos disponibles para cálculo de saturación")

## 7. Visualización Multipanel con PyPozo

PyPozo incluye un potente motor de visualización que permite crear gráficos multipanel profesionales para análisis petrofísico. Vamos a crear visualizaciones completas de nuestro pozo con todos los datos calculados.

In [ ]:
# Crear visualización multipanel básica
from pypozo.visualization.plotter import WellPlotter

# Inicializar el plotter
plotter = WellPlotter()

# Configurar panel básico con curvas principales
panel_config = {
    'GR': {'color': 'green', 'scale': 'linear'},
    'RT': {'color': 'red', 'scale': 'log'},
    'RHOB': {'color': 'blue', 'scale': 'linear'},
    'NPHI': {'color': 'cyan', 'scale': 'linear'}
}

# Crear el plot
fig = plotter.create_multipanel_plot(well, panel_config)
plt.show()

print("Visualización multipanel creada exitosamente")

In [ ]:
# Crear visualización avanzada con datos petrofísicos calculados
petro_panel_config = {
    'GR': {'color': 'green', 'title': 'Gamma Ray'},
    'VCL': {'color': 'brown', 'title': 'Volume de Arcilla'},
    'PHIE': {'color': 'blue', 'title': 'Porosidad Efectiva'},
    'SW': {'color': 'cyan', 'title': 'Saturación de Agua'},
    'RT': {'color': 'red', 'title': 'Resistividad', 'scale': 'log'}
}

# Crear plot petrofísico completo
try:
    # Verificar que tenemos los datos calculados en el well
    available_curves = list(well.data.columns)
    print("Curvas disponibles:", available_curves)
    
    # Crear plot con las curvas que tenemos
    fig, axes = plt.subplots(1, 5, figsize=(20, 12))
    
    # Plot GR
    if 'GR' in available_curves:
        axes[0].plot(well.data['GR'], well.data.index, 'g-', linewidth=1)
        axes[0].set_title('Gamma Ray (API)')
        axes[0].grid(True, alpha=0.3)
        axes[0].invert_yaxis()
    
    # Plot VCL si existe
    if 'VCL' in available_curves:
        axes[1].plot(well.data['VCL'], well.data.index, 'brown', linewidth=1)
        axes[1].set_title('Volume de Arcilla')
        axes[1].grid(True, alpha=0.3)
        axes[1].invert_yaxis()
    
    # Plot PHIE si existe
    if 'PHIE' in available_curves:
        axes[2].plot(well.data['PHIE'], well.data.index, 'b-', linewidth=1)
        axes[2].set_title('Porosidad Efectiva')
        axes[2].grid(True, alpha=0.3)
        axes[2].invert_yaxis()
    
    # Plot SW si existe
    if 'SW' in available_curves:
        axes[3].plot(well.data['SW'], well.data.index, 'c-', linewidth=1)
        axes[3].set_title('Saturación de Agua')
        axes[3].grid(True, alpha=0.3)
        axes[3].invert_yaxis()
    
    # Plot RT
    if 'RT' in available_curves:
        axes[4].semilogx(well.data['RT'], well.data.index, 'r-', linewidth=1)
        axes[4].set_title('Resistividad (ohm-m)')
        axes[4].grid(True, alpha=0.3)
        axes[4].invert_yaxis()
    
    plt.tight_layout()
    plt.suptitle(f'Análisis Petrofísico Completo - {well.name}', y=0.95, fontsize=16)
    plt.show()
    
    print("Visualización petrofísica completa creada exitosamente")
    
except Exception as e:
    print(f"Error en visualización: {e}")
    print("Creando plot básico alternativo...")
    
    # Plot básico alternativo
    fig, ax = plt.subplots(figsize=(8, 10))
    if 'GR' in available_curves:
        ax.plot(well.data['GR'], well.data.index, 'g-', label='GR')
    if 'RT' in available_curves:
        ax.plot(well.data['RT'], well.data.index, 'r-', label='RT')
    ax.invert_yaxis()
    ax.legend()
    ax.set_title(f'Curvas Básicas - {well.name}')
    plt.show()

## 8. Exportación de Resultados

PyPozo permite exportar tanto los datos procesados como las visualizaciones generadas en múltiples formatos para uso posterior en otros software de análisis petrofísico.

In [ ]:
# Exportar datos procesados a diferentes formatos
import os

# Crear carpeta de salida si no existe
output_dir = "../output_workflow_notebook"
os.makedirs(output_dir, exist_ok=True)

# 1. Exportar a archivo LAS con datos petrofísicos calculados
las_output_path = os.path.join(output_dir, f"{well.name}_procesado.las")
try:
    well.export_to_las(las_output_path)
    print(f"✓ Datos exportados a LAS: {las_output_path}")
except Exception as e:
    print(f"Error exportando LAS: {e}")

# 2. Exportar datos a CSV para análisis en Excel/Python
csv_output_path = os.path.join(output_dir, f"{well.name}_datos.csv")
try:
    well.data.to_csv(csv_output_path, index=True)
    print(f"✓ Datos exportados a CSV: {csv_output_path}")
except Exception as e:
    print(f"Error exportando CSV: {e}")

# 3. Exportar resumen estadístico
stats_output_path = os.path.join(output_dir, f"{well.name}_estadisticas.csv")
try:
    stats = well.data.describe()
    stats.to_csv(stats_output_path)
    print(f"✓ Estadísticas exportadas: {stats_output_path}")
    print("\nResumen estadístico:")
    print(stats)
except Exception as e:
    print(f"Error exportando estadísticas: {e}")

print(f"\nArchivos guardados en: {os.path.abspath(output_dir)}")

In [ ]:
# Exportar visualizaciones a archivos de imagen
print("\n=== Exportando Visualizaciones ===")

# Recrear y guardar la visualización petrofísica
fig, axes = plt.subplots(1, 4, figsize=(16, 12))

try:
    # Plot GR
    if 'GR' in well.data.columns:
        axes[0].plot(well.data['GR'], well.data.index, 'g-', linewidth=1)
        axes[0].set_title('Gamma Ray (API)')
        axes[0].grid(True, alpha=0.3)
        axes[0].invert_yaxis()
    
    # Plot VCL si existe
    if 'VCL' in well.data.columns:
        axes[1].plot(well.data['VCL'], well.data.index, 'brown', linewidth=1)
        axes[1].set_title('Volume de Arcilla')
        axes[1].grid(True, alpha=0.3)
        axes[1].invert_yaxis()
    
    # Plot PHIE si existe
    if 'PHIE' in well.data.columns:
        axes[2].plot(well.data['PHIE'], well.data.index, 'b-', linewidth=1)
        axes[2].set_title('Porosidad Efectiva')
        axes[2].grid(True, alpha=0.3)
        axes[2].invert_yaxis()
    
    # Plot SW si existe
    if 'SW' in well.data.columns:
        axes[3].plot(well.data['SW'], well.data.index, 'c-', linewidth=1)
        axes[3].set_title('Saturación de Agua')
        axes[3].grid(True, alpha=0.3)
        axes[3].invert_yaxis()
    
    plt.tight_layout()
    plt.suptitle(f'Análisis Petrofísico Completo - {well.name}', y=0.95, fontsize=16)
    
    # Guardar en diferentes formatos
    png_path = os.path.join(output_dir, f"{well.name}_analisis_completo.png")
    pdf_path = os.path.join(output_dir, f"{well.name}_analisis_completo.pdf")
    
    plt.savefig(png_path, dpi=300, bbox_inches='tight')
    plt.savefig(pdf_path, bbox_inches='tight')
    
    print(f"✓ Gráfico guardado en PNG: {png_path}")
    print(f"✓ Gráfico guardado en PDF: {pdf_path}")
    
    plt.show()
    
except Exception as e:
    print(f"Error generando gráficos: {e}")

# Crear gráfico de correlación si tenemos múltiples curvas
if len(well.data.columns) > 2:
    try:
        # Seleccionar curvas numéricas para correlación
        numeric_cols = well.data.select_dtypes(include=[np.number]).columns
        if len(numeric_cols) > 1:
            plt.figure(figsize=(10, 8))
            correlation_matrix = well.data[numeric_cols].corr()
            plt.imshow(correlation_matrix, cmap='coolwarm', aspect='auto')
            plt.colorbar(label='Correlación')
            plt.xticks(range(len(numeric_cols)), numeric_cols, rotation=45)
            plt.yticks(range(len(numeric_cols)), numeric_cols)
            plt.title(f'Matriz de Correlación - {well.name}')
            
            # Agregar valores de correlación
            for i in range(len(numeric_cols)):
                for j in range(len(numeric_cols)):
                    plt.text(j, i, f'{correlation_matrix.iloc[i,j]:.2f}', 
                            ha='center', va='center', color='black')
            
            corr_path = os.path.join(output_dir, f"{well.name}_correlacion.png")
            plt.savefig(corr_path, dpi=300, bbox_inches='tight')
            print(f"✓ Matriz de correlación guardada: {corr_path}")
            plt.show()
    except Exception as e:
        print(f"Error creando matriz de correlación: {e}")

print(f"\n🎉 Exportación completa. Revisa la carpeta: {os.path.abspath(output_dir)}")

## 9. Conclusiones y Siguientes Pasos

### ¿Qué hemos logrado?

En este notebook hemos demostrado el workflow completo de PyPozo 2.0:

✅ **Carga de datos LAS**: Importación robusta de archivos de registro de pozos  
✅ **Análisis de datos**: Inspección de metadatos y curvas disponibles  
✅ **Fusión de pozos**: Combinación inteligente de múltiples archivos LAS  
✅ **Cálculos petrofísicos**: Volume de arcilla, porosidad efectiva y saturación de agua  
✅ **Visualización profesional**: Gráficos multipanel para análisis integral  
✅ **Exportación completa**: Datos y gráficos en formatos estándar de la industria

### Capacidades avanzadas de PyPozo

PyPozo 2.0 incluye funcionalidades adicionales que puedes explorar:

- **Análisis de permeabilidad**: Cálculos usando correlaciones estándar
- **Identificación litológica**: Clasificación automática de facies
- **Análisis de crossplot**: Gráficos de dispersión para interpretación
- **Control de calidad**: Detección automática de valores anómalos
- **Integración con bases de datos**: Conexión con sistemas corporativos

### Próximos pasos recomendados

1. **Experimenta con tus propios datos**: Carga archivos LAS de tu proyecto
2. **Personaliza los cálculos**: Ajusta parámetros según tu yacimiento
3. **Explora la GUI**: Utiliza `pypozo_app.py` para análisis interactivo
4. **Automatiza workflows**: Crea scripts para procesamiento en lote
5. **Contribuye al proyecto**: Reporta issues y sugiere mejoras en GitHub

### Recursos adicionales

- **Documentación completa**: `/docs/MANUAL_USUARIO.md`
- **Guía rápida**: `/docs/GUIA_RAPIDA.md`
- **Ejemplos adicionales**: `/examples/` y `/demo/`
- **API Reference**: `/docs/API_REFERENCE.md`

---

**¡Gracias por usar PyPozo 2.0!**  
*Tu herramienta Python para análisis petrofísico moderno*